In [136]:
import pandas as pd
import seaborn as sns
import math
import numpy as np
from collections import Counter
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler


In [164]:
#imputo la moda

def imputo_moda(df):
    mode_imputer = SimpleImputer(strategy='most_frequent')
    df_2 = df.copy()
    df_2.iloc[:,:] = mode_imputer.fit_transform(df_2)
    return(df_2)

#One Hot Encoding

def OHE(df,lista_colonne):

    for nome_colonna in lista_colonne:
        df_onehot_encoded = pd.get_dummies(df[nome_colonna])
        df = pd.concat([df, df_onehot_encoded], axis=1)

    df.drop(labels = lista_colonne, inplace=True,axis=1)
    return(df)
    
#trasformo true/falso in 0/1

def TrueFalsetoInt(df,lista_colonne):

    for nome_colonna in lista_colonne:
        df[nome_colonna] = df[nome_colonna]*1
    return(df)

In [165]:
df_train = pd.read_csv("train.csv")


In [166]:
#Splitto la cabina

df_cabin = df_train['Cabin'].str.split("/",expand=True)
df_cabin.columns = ['deck', 'num', 'side']

df_train = pd.concat([df_train, df_cabin], axis=1)
df_train.drop(labels = 'Cabin', inplace=True,axis=1)

In [167]:
#Imputo tutto con la moda
df_train_2 = imputo_moda(df_train)

In [168]:
def categorical_variable(df,feature_name,type_eng):

    if type_eng=="One-Hot-Encoding":
        df_onehot_encoded = pd.get_dummies(df[feature_name])

        df_new = pd.concat([df, df_onehot_encoded], axis=1)
        df_new.drop(labels = [feature_name], inplace=True,axis=1)
        return(df_new)

    if type_eng=="One-Hot-Encoding-one-less":

        df_onehot_encoded = pd.get_dummies(df[feature_name]).iloc[: , 1:] #ignora la prima colonna creata

        df_new = pd.concat([df, df_onehot_encoded], axis=1)
        df_new.drop(labels = [feature_name], inplace=True,axis=1)
        return(df_new)
    
    if type_eng=="Label-encoding":

        labelencoder = LabelEncoder()

        df_new=df.copy()
        df_new[feature_name] = labelencoder.fit_transform(df_new[feature_name])
        return(df_new)
    
    if type_eng=="Categorical":

        return(df)


In [169]:
def numerical_variable(df, feature_name, type_eng, type_sub_eng, category_boundaries):

    if type_eng=="Numerical":

        return(df)
    
    if type_eng=="Standardization":
        scaler = StandardScaler()
        df_new=df.copy()
        df_new[feature_name] = scaler.fit_transform(df_new[feature_name])
        return(df_new)

    if type_eng=="Min-max":
        scaler = MinMaxScaler()
        df_new=df.copy()
        df_new[feature_name] = scaler.fit_transform(df_new[feature_name])
        return(df_new)    
    
    if type_eng=="Category":
        df_new = df.copy()
        labels = [*range(len(category_boundaries)-1)]
        labels = [feature_name+"_"+str(s)  for s in labels]
        df_new[feature_name] = pd.cut(df_new[feature_name],category_boundaries,labels=labels,include_lowest=True)

        return(categorical_variable(df_new,feature_name,type_sub_eng))   
            



In [170]:
variabili_categoriche = ["HomePlanet","CryoSleep","Destination","VIP","side","deck"]
variabili_numeriche = ["Age","RoomService","FoodCourt","ShoppingMall","Spa","VRDeck"]
variabili_inutili = ['PassengerId','Name','num']

In [171]:
#Elimino le variabili inutili

df_train_2.drop(labels = ['PassengerId','Name','num'], inplace=True,axis=1)

#Eseguo tutte le combinazioni di lavorazioni che ha senso fare

lavorazioni_categoriche = ["One-Hot-Encoding","One-Hot-Encoding-one-less","Label-encoding"]
lavorazioni_numeriche = ["Numerical","Standardization","Min-max",
                        #["Category","One-Hot-Encoding"],
                        #["Category","One-Hot-Encoding-one-less"],
                        #["Category","Label-encoding"],
                        ]


In [172]:
#Lista Lavorazioni

from itertools import product

A=[]
for i in range(len(variabili_categoriche)-1):
    if i==0:
        A=list(product(lavorazioni_categoriche,lavorazioni_categoriche))
    else:
        A=list(product(A,lavorazioni_categoriche))
        A=[(*x,z) for x, z in A]

for i in range(len(variabili_numeriche)):
    print(len(A),i)    
    A=list(product(A,lavorazioni_numeriche))
    A=[(*x,z) for x, z in A]

df_lavorazioni = pd.DataFrame(A,columns=variabili_categoriche+variabili_numeriche)


729 0
2187 1
6561 2
19683 3
59049 4
177147 5


In [173]:
#Adaline


In [174]:
def net_input(w_, X):
    """Calculate net input"""
    return np.dot(X, w_[1:]) + w_[0]

def activation(self, X):
    """Compute linear activation"""
    return X

def predict(self, X):
    """Return class label after unit step"""
    return np.where(self.activation(self.net_input(X)) >= 0.0, 1, -1)

In [175]:
#creo i vari dataframe e testo il modello

n_iter=2000
eta= 0.0000001
random_state=1

y_train = df_train_2["Transported"].replace(0,-1)
X_train = df_train_2.drop(columns=["Transported"])


In [181]:
from sklearn.model_selection import train_test_split



for row in range(len(df_lavorazioni)):
    
    df=df_train_2.copy()


    for feature in df_lavorazioni.columns:

        lavorazione = df_lavorazioni.loc[0,feature]
        if feature in variabili_categoriche:
            df=categorical_variable(df,feature,lavorazione)
        else:
            df=numerical_variable(df,feature,lavorazione,1,1)
            df =numerical_variable(df,feature,lavorazione,1,1)              


    df_train_row, df_test_row = train_test_split(df, test_size=0.3,random_state=1)

    X_train = df_train_row.drop(columns=["Transported"])
    y_train = 1*df_train_row["Transported"].replace(0,-1)


    X_test = df_test_row.drop(columns=["Transported"])
    y_test = 1*df_test_row["Transported"].replace(0,-1)


    rgen = np.random.RandomState(random_state)
    w_ = rgen.normal(loc=0.0, scale=0.01, size= 1 + X_train.shape[1])
    cost_ = []

    for i in range(n_iter):
        output = net_input(w_ ,X_train)
        errors = (y_train - output)
        w_[1:] += eta * X_train.T.dot(errors)
        w_[0] += eta * errors.sum()
        cost = (errors**2).sum() / 2.0
        cost_.append(cost)


    print(sum((np.where(net_input(w_,X_test) >= 0.0, 1, -1)==y_test)*1)/len(np.where(net_input(w_,X_test) >= 0.0, 1, -1)-y_test))





        
    break

0.0


In [182]:
y_test

1454    1
218     0
7866    0
7622    1
4108    0
       ..
1930    0
7620    1
7503    1
6323    1
4204    1
Name: Transported, Length: 2608, dtype: int32

In [162]:
df_train_row.columns

Index([          'Age',   'RoomService',     'FoodCourt',  'ShoppingMall',
                 'Spa',        'VRDeck',         'Earth',        'Europa',
                'Mars',           False,            True,   '55 Cancri e',
       'PSO J318.5-22',   'TRAPPIST-1e',           False,            True,
                   'P',             'S',             'A',             'B',
                   'C',             'D',             'E',             'F',
                   'G',             'T'],
      dtype='object')

In [163]:
df_test_row.columns

Index([          'Age',   'RoomService',     'FoodCourt',  'ShoppingMall',
                 'Spa',        'VRDeck',         'Earth',        'Europa',
                'Mars',           False,            True,   '55 Cancri e',
       'PSO J318.5-22',   'TRAPPIST-1e',           False,            True,
                   'P',             'S',             'A',             'B',
                   'C',             'D',             'E',             'F',
                   'G'],
      dtype='object')

In [ ]:


# one-hot-encoding di HomePlanet // Destination
#Destination

df_train_2 = OHE(df_train_2,['HomePlanet','Destination','side','deck'])
df_test_2 = OHE(df_test_2,['HomePlanet','Destination','side','deck'])


#trasformo True/False in 1/0

df_train_2 = TrueFalsetoInt(df_train_2,['CryoSleep','VIP','Transported'])
df_test_2 = TrueFalsetoInt(df_test_2,['CryoSleep','VIP'])


#Tolgo le colonne che non mi servono o non so come usare ancora



In [ ]:
df_train_2.to_excel('train_1.xlsx')
df_test_2.to_excel('test_1.xlsx')

In [ ]:

print(i)

5
